[View in Colaboratory](https://colab.research.google.com/github/SeaGoUnlimited/SB-Water-Quality/blob/master/spotter.ipynb)

In [35]:
! pip install geopandas

Importing the necessary modules. 

In [0]:
import os, json
import pandas 
import numpy as np
import geopandas
from shapely.geometry import Point, LineString

Loading json file, extracting the track data, and converting it into a geodataframe.

In [0]:
#Create a function that loads json by filename
def loadjson(filename):
  with open(filename) as f:
    data = json.load(f)
  observer= data['Observer Names']
  date=data['create_date']
  sightings = extract_sightings(data)   
  track = extract_track(data)
  del data['track']
  del data['sightings']  
  data['track'] = track  
  df = pandas.DataFrame(data)
  trip = geopandas.GeoDataFrame(df, geometry='track')  
  return sightings, trip




Turning the track data into a linestring.

In [0]:
def extract_track(data):
  coordinates = []
  for trkpt in data['track']['gpx']['trk']['trkseg']['trkpt']:
    coordinate = (float(trkpt['@lon']), float(trkpt['@lat']))
    coordinates.append(coordinate)
  line = LineString(coordinates).wkt
  return line


Extracting the sightings data  and putting it into a pandas dataframe. Then created a new datafram which is merged by date. This dataframe is transformed to a point which is stored in the new column, 'coordinates'. Then the dataframe is converted to a geodataframe.

In [0]:
#Create a function to extract sightings data
def extract_sightings(data):
  sightings = data['sightings']
  df = pandas.DataFrame()
  for sighting in sightings:
    df = df.append(sighting, ignore_index=True)    
    #create a new data frame merged by date, pulling all the first occurances in the new table
  df = df.groupby(by='create_date').first()
  df['coordinates'] =  list(zip(df.lon, df.lat))
  df['coordinates'] = df['coordinates'].apply(Point)
  gdf = geopandas.GeoDataFrame(df, geometry='coordinates')
  return gdf

Main function needed to create a loop.

In [86]:
#This is the main function
sightings, track = loadjson(filename="trip_58.json")
track

LINESTRING (-122.504008 37.656192, -122.504068 37.656205, -122.504381 37.656163, -122.504245 37.656216, -122.504325 37.656133, -122.504501 37.656057, -122.504643 37.656076, -122.504729 37.656068, -122.504753 37.656121, -122.504915 37.656056, -122.50534 37.656016, -122.505801 37.655984, -122.505929 37.655979, -122.506047 37.655998, -122.506162 37.655977, -122.506279 37.655969, -122.50687 37.655962, -122.506977 37.655944, -122.50708 37.655957, -122.507353 37.65595, -122.507917 37.655915, -122.508219 37.655922, -122.508643 37.655919, -122.508766 37.655915, -122.509142 37.655904, -122.509356 37.655884, -122.509873 37.655862, -122.510108 37.655882, -122.510516 37.655877, -122.51116 37.65589, -122.511383 37.655879, -122.512783 37.655875, -122.513017 37.655884, -122.513247 37.655906, -122.513371 37.655905, -122.513604 37.655917, -122.51366 37.655914, -122.513902 37.655919, -122.514036 37.655933, -122.514148 37.655905, -122.514304 37.655892, -122.514355 37.655908, -122.514476 37.65589, -122.51

TypeError: ignored